In [16]:
from twitter_scraper import get_tweets
import pandas as pd
import re
import time

from sqlalchemy import create_engine


In [2]:
# selected_tag = ['#CoronaOutbreak,#CoronavirusOutbreak,#coronavirus', '#COVID19']

res_list = []
result_df = pd.DataFrame()
keyword_list = ['#Covid19','KKMPutrajaya'] ## other keyword such as #CoronaVirus, #covid19
num_pages = 10

for kw in keyword_list:
    for i in range(1, num_pages+1):
        for tweet in get_tweets( kw , pages=i):
            tweet['time'] = str( tweet['time'])  ## convert datetime object to str 
            tweet['query_keywords'] = str(kw) 
            res_list.append(tweet)
        time.sleep(1) ## to avoid api call too frequent

In [3]:
res_list


[{'tweetId': '1243333034184769536',
  'isRetweet': False,
  'time': '2020-03-27 08:24:30',
  'text': 'La propuesta de @alferdez Durante la videoconferencia con líderes del G20 para la creacion de un #FondoMundialdeEmergencia Humanitaria contra el #COVID19 es una propuesta que puede fortalecer la salud mundial y posicionar a nuestro pais mundialmente.',
  'replies': 0,
  'retweets': 0,
  'likes': 0,
  'entries': {'hashtags': ['#FondoMundialdeEmergencia', '#COVID19'],
   'urls': [],
   'photos': [],
   'videos': []},
  'query_keywords': '#Covid19'},
 {'tweetId': '1243333033794523137',
  'isRetweet': False,
  'time': '2020-03-27 08:24:29',
  'text': 'And Trump wants to position #UStroops along the US-Canadian border to prevent ppl from crossing into the US illegally, when #Canada has universal healthcare, compassion resources and far less infections of #COVID19 .\n#TrumpMadness',
  'replies': 0,
  'retweets': 0,
  'likes': 0,
  'entries': {'hashtags': ['#UStroops',
    '#Canada',
    '#CO

In [4]:
len(res_list)

2201

In [5]:
df = pd.DataFrame(res_list) 


In [6]:
df.head()

,tweetId,isRetweet,time,text,replies,retweets,likes,entries,query_keywords
0,1243333034184769536,False,2020-03-27 08:24:30,La propuesta de @alferdez Durante la videoconf...,0,0,0,"{'hashtags': ['#FondoMundialdeEmergencia', '#C...",#Covid19
1,1243333033794523137,False,2020-03-27 08:24:29,And Trump wants to position #UStroops along th...,0,0,0,"{'hashtags': ['#UStroops', '#Canada', '#COVID1...",#Covid19
2,1243333031542185984,False,2020-03-27 08:24:29,Amalkan langkah-langkah pencegahan penularan C...,0,0,0,"{'hashtags': ['#COVID19', '#DudukRumah', '#Sta...",#Covid19
3,1243333031508705286,False,2020-03-27 08:24:29,This Scarborough guy is nuts...it is impossibl...,0,0,0,"{'hashtags': ['#COVID19'], 'urls': [], 'photos...",#Covid19
4,1243333028602089473,False,2020-03-27 08:24:28,"On Jan. 31, Trump announced a travel ban on Ch...",0,0,0,"{'hashtags': ['#COVID19', '#COVID19'], 'urls':...",#Covid19


## data extraction

In [94]:
def extract_kes_kematian( text ):
    ## single string pattern match (return whole sentence left and right) [^.]*?Text[^.]*\.
    ## multiple string pattern match (separate by |)  :  [^.]*?Text1[^.]*\.|[^.]*?Text2[^.]*\.
    
    result = re.search(r"(kematian ke-(\d+))", str(text) )

    return ",".join(str(result)) ## join 

def extract_kes_kematian_info( text ):
    ## single string pattern match (return whole sentence left and right) [^.]*?Text[^.]*\.
    ## multiple string pattern match (separate by |)  :  [^.]*?Text1[^.]*\.|[^.]*?Text2[^.]*\.
    
    result = re.findall(r"(kematian ke-[^.]*\.)", str(text) )  

    return ",".join(result) ## join 


def extract_kes_pesakit( text ):
    ## single string pattern match (return whole sentence left and right) [^.]*?Text[^.]*\.
    ## multiple string pattern match (separate by |)  :  [^.]*?Text1[^.]*\.|[^.]*?Text2[^.]*\.
    
    result = re.findall(r"(pesakit ke-(\d+))", str(text) )  

    return ",".join(result) ## join 

def extract_kes_pesakit_info( text ):
    ## single string pattern match (return whole sentence left and right) [^.]*?Text[^.]*\.
    ## multiple string pattern match (separate by |)  :  [^.]*?Text1[^.]*\.|[^.]*?Text2[^.]*\.
    
    result = re.findall(r"(pesakit ke-[^.]*\.)", str(text) )  

    return ",".join(result) ## join 



def extract_age( text ):
    ## single string pattern match (return whole sentence left and right) [^.]*?Text[^.]*\.
    ## multiple string pattern match (separate by |)  :  [^.]*?Text1[^.]*\.|[^.]*?Text2[^.]*\.
    
    result = re.findone(r"(umur (\d+) )", str(text) )  

    return ",".join(result) ## join 


def entity_tag(article):
    doc = nlp(article.encode('ascii', 'ignore').decode('utf8') )   ## handling text unicode 
    label = []
    text = []
    full=[]
    entity_list = ['PERSON','ORG','GPE','PRODUCT']
    for ent, i in zip(doc.ents,range(0,len(doc))):
        label.append(ent.label_)
        text.append(ent.text)
        if label[i] in entity_list:
            full.append(text[i])
    
    entity = list(set(full))
    return entity




## save file into json & csv

In [7]:
import json


with open('corona_tweet.json', 'w') as file:
    json.dump(res_list,file)

In [15]:
df.to_csv('corona_tweet.csv',index=False)

## save to mySQL db

In [ ]:
## to-be-update

In [13]:

## DB connection
sql_hostname = '' ## YOUR HOST_IP
sql_username = '' 
sql_password = ''
sql_main_database = 'coronatracker'
sql_table ="twitter_tweet"  ## create table name


# engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(sql_username,sql_password,sql_hostname,sql_main_database))

engine = create_engine('mysql://{}:{}@{}/{}??charset=ut8'.format(sql_username,sql_password,sql_hostname,sql_main_database))


## SQL query
query = "select * from " + str(sql_main_database) + "." + str(sql_table)

In [130]:
## encode the text
# df['text'] = df['text'].str.encode('utf-8')

TypeError: Cannot use .str.encode with values of inferred dtype 'bytes'.

In [10]:
df['text'] 

0       La propuesta de @alferdez Durante la videoconf...
1       And Trump wants to position #UStroops along th...
2       Amalkan langkah-langkah pencegahan penularan C...
3       This Scarborough guy is nuts...it is impossibl...
4       On Jan. 31, Trump announced a travel ban on Ch...
                              ...                        
2196    Walaupun tinggal di dalam rumah, rakyat Malays...
2197    The ‘Corporate Malaysia Against COVID-19 Actio...
2198    The Ministry of Health Malaysia (MOH), togethe...
2199    Pecahan keseluruhan 1,030 kes di Malaysia seta...
2200    Ringkasan situasi semasa penularan jangkitan #...
Name: text, Length: 2201, dtype: object

In [14]:

## save the file to db
df.to_sql(   str((sql_table)), con = engine,  if_exists='append',index=False)

UnicodeEncodeError: 'charmap' codec can't encode character '\u2066' in position 109: character maps to <undefined>